In [6]:
import pandas as pd
from UTILS import Time_Converter, Geographic

In [4]:
path = "London Datathon Materials Anonymized/"
df = pd.read_csv(path + "dataset_9.csv")

In [7]:
Geographic.lati_long_converter(df["BK88"])

,latitude,longitude
0,-73.976051,40.631284
1,-73.977167,40.630755
2,-73.977000,40.629880
3,-73.976851,40.629097
4,-73.976697,NaN
